In [2]:
import dask.dataframe as dd
from sklearn.model_selection import train_test_split
from dask_ml.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

ddf: dd = dd.read_parquet('../../data/merge/mp_current_new_feat.parquet')

In [ ]:
ddf.reset_index(inplace=True)
ddf.drop(['latitude', 'longitude', 'ug', 'vg'], axis=1, inplace=True)
ddf = ddf[ddf['u'].isnull() == False]
ddf = ddf[ddf['u'] != -999.0]
ddf['time'] = ddf['time'].astype('string')
ddf['time'] = ddf.to_datetime(ddf['time'], errors='coerce')
ddf: dd = ddf.dropna(subset=['time'])

In [ ]:
colsFloat = ['lat', 'lon', 'u', 'v']
for col in colsFloat:
    ddf[col] = ddf[col].astype('float16')

In [ ]:
## encode oceans using one-hot encoding (dask)
ddf['ocean'] = ddf['ocean'].astype('category')
enc = OneHotEncoder(handle_unknown='ignore')
ddf['ocean'] = enc.fit_transform(ddf['ocean'].values.reshape(-1, 1))

In [ ]:
df = ddf.compute()

In [ ]:
cols = ['lat', 'lon', 'num_MP_samples', 'u', 'v', 'ug', 'vg', 'ocean_0', 'ocean_1', 'ocean_2']
xTrain, xTest, yTrain, yTest = train_test_split(df[cols], df["MP_concentration"], test_size=0.2, random_state=0)

In [ ]:
model = xgb.XGBRegressor()
model.fit(xTrain, yTrain)
predicao = model.predict(xTest)

In [ ]:
mae = mean_absolute_error(predicao, yTest)
mae